### Given a Bank customer, can we build a classifier which can determine whether they will leave or not using Neural networks?

### Load tensorflow

In [1]:
%tensorflow_version 2.x
import tensorflow as tf
import numpy as np
import pandas as pd


TensorFlow 2.x selected.


In [2]:
tf.__version__

'2.1.0-rc1'

### Read the dataset in a new python notebook

In [3]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


In [0]:
df_bank = pd.read_csv("/content/drive/My Drive/PGPAIML/30Nov/Assignment/Project-1/bank.csv")

In [5]:
df_bank.head(5)

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [6]:
df_bank.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
RowNumber          10000 non-null int64
CustomerId         10000 non-null int64
Surname            10000 non-null object
CreditScore        10000 non-null int64
Geography          10000 non-null object
Gender             10000 non-null object
Age                10000 non-null int64
Tenure             10000 non-null int64
Balance            10000 non-null float64
NumOfProducts      10000 non-null int64
HasCrCard          10000 non-null int64
IsActiveMember     10000 non-null int64
EstimatedSalary    10000 non-null float64
Exited             10000 non-null int64
dtypes: float64(2), int64(9), object(3)
memory usage: 1.1+ MB


### Drop the columns which are unique for all users like IDs 

In [7]:
df_bank=df_bank.drop(['RowNumber','CustomerId','Surname'],axis=1)
df_bank.head(5)

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [0]:

from sklearn.preprocessing import LabelEncoder,OneHotEncoder
# filter categorical columns using mask and turn it into a list
categorical_cols = df_bank.select_dtypes(exclude = ['int64','float64']).columns.tolist()

In [9]:
categorical_cols

['Geography', 'Gender']

In [10]:
# instantiate labelencoder  
le = LabelEncoder()
# apply le on categorical feature columns
df_bank[categorical_cols] = df_bank[categorical_cols].apply(lambda col: le.fit_transform(col))
df_bank.head(10)

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,0,0,42,2,0.00,1,1,1,101348.88,1
1,608,2,0,41,1,83807.86,1,0,1,112542.58,0
2,502,0,0,42,8,159660.80,3,1,0,113931.57,1
3,699,0,0,39,1,0.00,2,0,0,93826.63,0
4,850,2,0,43,2,125510.82,1,1,1,79084.10,0
5,645,2,1,44,8,113755.78,2,1,0,149756.71,1
6,822,0,1,50,7,0.00,2,1,1,10062.80,0
7,376,1,0,29,4,115046.74,4,1,0,119346.88,1
8,501,0,1,44,4,142051.07,2,0,1,74940.50,0
9,684,0,1,27,2,134603.88,1,1,1,71725.73,0


###Distinguish the feature and target set 

In [0]:
X=df_bank.drop('Exited',axis=1)
Y=df_bank['Exited']


In [98]:
X.head(5)

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,619,0,0,42,2,0.00,1,1,1,101348.88
1,608,2,0,41,1,83807.86,1,0,1,112542.58
2,502,0,0,42,8,159660.80,3,1,0,113931.57
3,699,0,0,39,1,0.00,2,0,0,93826.63
4,850,2,0,43,2,125510.82,1,1,1,79084.10


In [99]:
onehotencoder=OneHotEncoder(categorical_features=[1])
X=onehotencoder.fit_transform(X).toarray()

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_encoders.py:451: DeprecationWarning: The 'categorical_features' keyword is deprecated in version 0.20 and will be removed in 0.22. You can use the ColumnTransformer instead.
  "use the ColumnTransformer instead.", DeprecationWarning)


###Divide the data set into Train and test sets

In [0]:
from sklearn.model_selection import train_test_split
train_x,test_x,train_y,test_y=train_test_split(X,Y,test_size=.20,random_state=0)

In [0]:
train_y_cat = tf.keras.utils.to_categorical(train_y, num_classes=None)
test_y_cat = tf.keras.utils.to_categorical(test_y, num_classes=None)

In [0]:
train_x =np.array(train_x).astype('float32')
test_x = np.array(test_x).astype('float32')
train_y_cat =np.array(train_y_cat).astype('float32')
test_y_cat = np.array(test_y_cat).astype('float32')

In [103]:
train_x.shape

(8000, 12)

In [104]:
train_y_cat

array([[1., 0.],
       [1., 0.],
       [1., 0.],
       ...,
       [1., 0.],
       [1., 0.],
       [0., 1.]], dtype=float32)

In [105]:
test_x.shape

(2000, 12)

In [106]:
test_y_cat.shape

(2000, 2)

###Normalize the train and test data 

In [0]:
from sklearn.preprocessing import StandardScaler
sc=StandardScaler()
normalized_train_x = sc.fit_transform(train_x)
normalized_test_x = sc.transform(test_x)

###Initialize &amp; build the model

In [108]:
normalized_train_x.shape

(8000, 12)

In [0]:

tf.keras.backend.clear_session()
model1 = tf.keras.Sequential([tf.keras.layers.Flatten(input_shape=(12,)),
                              tf.keras.layers.BatchNormalization(),                             
                              tf.keras.layers.Dense(100,activation='sigmoid'),                              
    tf.keras.layers.Dense(2, activation=tf.nn.softmax)
])

In [0]:
model1.compile(optimizer='SGD',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [114]:
model1.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 12)                0         
_________________________________________________________________
batch_normalization (BatchNo (None, 12)                48        
_________________________________________________________________
dense (Dense)                (None, 100)               1300      
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 202       
Total params: 1,550
Trainable params: 1,526
Non-trainable params: 24
_________________________________________________________________


In [115]:
model1.fit(normalized_train_x, train_y_cat, validation_data=(normalized_test_x, test_y_cat), epochs=100,
          batch_size = 10)

Train on 8000 samples, validate on 2000 samples
Epoch 1/100
8000/8000 [==============================] - 2s 199us/sample - loss: 0.4831 - accuracy: 0.7960 - val_loss: 0.4646 - val_accuracy: 0.7975
Epoch 2/100
8000/8000 [==============================] - 1s 156us/sample - loss: 0.4448 - accuracy: 0.8012 - val_loss: 0.4407 - val_accuracy: 0.8015
Epoch 3/100
8000/8000 [==============================] - 1s 171us/sample - loss: 0.4362 - accuracy: 0.8102 - val_loss: 0.4290 - val_accuracy: 0.8115
Epoch 4/100
8000/8000 [==============================] - 1s 171us/sample - loss: 0.4310 - accuracy: 0.8133 - val_loss: 0.4310 - val_accuracy: 0.8080
Epoch 5/100
8000/8000 [==============================] - 1s 165us/sample - loss: 0.4346 - accuracy: 0.8090 - val_loss: 0.4276 - val_accuracy: 0.8130
Epoch 6/100
8000/8000 [==============================] - 1s 165us/sample - loss: 0.4330 - accuracy: 0.8111 - val_loss: 0.4277 - val_accuracy: 0.8145
Epoch 7/100
8000/8000 [==============================] - 1

In [116]:
#Predicting the Test set results
y_pred = model1.predict_classes(normalized_test_x)
y_pred = (y_pred > .5)
y_pred

array([False, False, False, ..., False, False, False])

In [117]:
#Making the COnfusion Matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(test_y, y_pred)
cm

array([[1504,   91],
       [ 232,  173]])

In [0]:
tf.keras.backend.clear_session()
#Initialize model, reshape & normalize data
model2 = tf.keras.models.Sequential()

model2.add(tf.keras.layers.Flatten(input_shape=(12,)))

#normalize data
model2.add(tf.keras.layers.BatchNormalization())
model2.add(tf.keras.layers.Dropout(0.2))

#Add 1st hidden layer
model2.add(tf.keras.layers.Dense(200, activation='relu'))
model2.add(tf.keras.layers.Dropout(0.2))

#Add 2nd hidden layer
model2.add(tf.keras.layers.Dense(200, activation='relu'))
#Add another dropout layer


In [0]:
#Add OUTPUT layer
model2.add(tf.keras.layers.Dense(2, activation='softmax'))

In [0]:
model2.compile(optimizer='adam', loss='categorical_crossentropy', 
              metrics=['accuracy'])

In [128]:
model2.fit(normalized_train_x, train_y_cat, validation_data=(normalized_test_x, test_y_cat),        
          epochs=100,
          batch_size=10)

Train on 8000 samples, validate on 2000 samples
Epoch 1/100
8000/8000 [==============================] - 2s 268us/sample - loss: 0.4593 - accuracy: 0.8026 - val_loss: 0.4062 - val_accuracy: 0.8360
Epoch 2/100
8000/8000 [==============================] - 2s 209us/sample - loss: 0.4296 - accuracy: 0.8135 - val_loss: 0.3756 - val_accuracy: 0.8540
Epoch 3/100
8000/8000 [==============================] - 2s 203us/sample - loss: 0.4241 - accuracy: 0.8163 - val_loss: 0.3576 - val_accuracy: 0.8600
Epoch 4/100
8000/8000 [==============================] - 2s 207us/sample - loss: 0.4151 - accuracy: 0.8199 - val_loss: 0.3588 - val_accuracy: 0.8545
Epoch 5/100
8000/8000 [==============================] - 2s 205us/sample - loss: 0.4158 - accuracy: 0.8214 - val_loss: 0.3588 - val_accuracy: 0.8585
Epoch 6/100
8000/8000 [==============================] - 2s 203us/sample - loss: 0.4130 - accuracy: 0.8236 - val_loss: 0.3553 - val_accuracy: 0.8570
Epoch 7/100
8000/8000 [==============================] - 2

In [130]:
#Predicting the Test set results
y_pred = model2.predict_classes(normalized_test_x)
y_pred = (y_pred > .5)
y_pred

array([False, False, False, ..., False, False, False])

In [131]:
#Making the COnfusion Matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(test_y, y_pred)
cm

array([[1517,   78],
       [ 201,  204]])

In [0]:
##SO after optimization accuracy got increaded very little but it got increased .